In [1]:
import torch
import torch.nn as nn
from model import LocalPredictor

In [2]:
import os

In [3]:
import pickle as pk

In [4]:
import numpy as np

In [5]:
local_predictor = torch.load('./results_osaka/local_predictor_fl_scratch_noise001.pytorch').cuda(0)

In [6]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [7]:
T = 96
dT = 4
batch_size = 256

In [8]:
data = dict({})
for m in range(11, 12):
    for d in range(1, 29):
        filename = '/data/fan/UsersInOsakaProcessed/2012{:02d}{:02d}_interp.pk'.format(m, d)
        print(filename)
        with open(filename, 'rb') as f:
            data[(m, d)] = pk.load(f)

/data/fan/UsersInOsakaProcessed/20121101_interp.pk
/data/fan/UsersInOsakaProcessed/20121102_interp.pk
/data/fan/UsersInOsakaProcessed/20121103_interp.pk
/data/fan/UsersInOsakaProcessed/20121104_interp.pk
/data/fan/UsersInOsakaProcessed/20121105_interp.pk
/data/fan/UsersInOsakaProcessed/20121106_interp.pk
/data/fan/UsersInOsakaProcessed/20121107_interp.pk
/data/fan/UsersInOsakaProcessed/20121108_interp.pk
/data/fan/UsersInOsakaProcessed/20121109_interp.pk
/data/fan/UsersInOsakaProcessed/20121110_interp.pk
/data/fan/UsersInOsakaProcessed/20121111_interp.pk
/data/fan/UsersInOsakaProcessed/20121112_interp.pk
/data/fan/UsersInOsakaProcessed/20121113_interp.pk
/data/fan/UsersInOsakaProcessed/20121114_interp.pk
/data/fan/UsersInOsakaProcessed/20121115_interp.pk
/data/fan/UsersInOsakaProcessed/20121116_interp.pk
/data/fan/UsersInOsakaProcessed/20121117_interp.pk
/data/fan/UsersInOsakaProcessed/20121118_interp.pk
/data/fan/UsersInOsakaProcessed/20121119_interp.pk
/data/fan/UsersInOsakaProcessed

In [9]:
data_doc = dict({})

for date in data:
    for uid in data[date]:
        if uid not in data_doc:
            data_doc[uid] = [data[date][uid]]
        else:
            data_doc[uid].append(data[date][uid])

In [10]:
for uid in data_doc:
    data_doc[uid] = torch.LongTensor(data_doc[uid]).cuda(0)

In [11]:
mrr_avg_rank_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInOsakaProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)
    
    with open(filename, 'rb') as f:
        cnt_day = 0
        data = pk.load(f)
        for uid in data:
            data[uid] = torch.LongTensor([data[uid]]).cuda(0)
            
        mrr = 0.0
        avg_rank = 0.0
        acc5 = 0.0
        
        with torch.no_grad():
            for t in range(T - 2 * dT):
                
                for uid in data:
                    if np.random.ranf() < 0.05:
                        cnt_day += 1
                        data_x = data[uid][:, t: t + dT]
                        data_t = torch.zeros_like(data_x) + t
                        data_y = data[uid][:, t + 2 * dT - 1].cpu().numpy()
                        if uid in data_doc:
                            data_doc_x = data_doc[uid][:, t: t + 2 * dT]
                            data_doc_t = torch.zeros_like(data_doc_x) + t
                            pred = local_predictor.predict(data_x, data_t, data_doc_x, data_doc_t).cpu().numpy()
                        else:
                            pred = local_predictor.predict(data_x, data_t, None, None).cpu().numpy()
                            
                        rank = np.where((np.argsort(-pred, axis=1, ).T - data_y).T == 0)[1] + 1
                        mrr += np.sum(1 / rank)
                        avg_rank += np.sum(rank)
                        acc5 += np.sum(rank <= 5)
                        
            print('Evaluation: mrr = {:.4f}, avg_rank = {:.4f}, acc@5 = {:.4f}'.format(mrr / cnt_day, avg_rank / cnt_day, acc5 / cnt_day))
            mrr_avg_rank_dict[(y, m, d)] = (mrr / cnt_day, avg_rank / cnt_day, acc5 / cnt_day)

/data/fan/UsersInOsakaProcessed/20121201_interp.pk
Evaluation: mrr = 0.8153, avg_rank = 17.0239, acc@5 = 0.8771
/data/fan/UsersInOsakaProcessed/20121202_interp.pk
Evaluation: mrr = 0.8241, avg_rank = 16.0747, acc@5 = 0.8822
/data/fan/UsersInOsakaProcessed/20121203_interp.pk
Evaluation: mrr = 0.8294, avg_rank = 13.8690, acc@5 = 0.8926
/data/fan/UsersInOsakaProcessed/20121204_interp.pk
Evaluation: mrr = 0.8292, avg_rank = 13.8387, acc@5 = 0.8924
/data/fan/UsersInOsakaProcessed/20121205_interp.pk
Evaluation: mrr = 0.8263, avg_rank = 14.7874, acc@5 = 0.8889
/data/fan/UsersInOsakaProcessed/20121206_interp.pk
Evaluation: mrr = 0.8249, avg_rank = 14.4963, acc@5 = 0.8885
/data/fan/UsersInOsakaProcessed/20121207_interp.pk
Evaluation: mrr = 0.8185, avg_rank = 15.2005, acc@5 = 0.8831
/data/fan/UsersInOsakaProcessed/20121208_interp.pk
Evaluation: mrr = 0.8143, avg_rank = 17.0496, acc@5 = 0.8756
/data/fan/UsersInOsakaProcessed/20121209_interp.pk
Evaluation: mrr = 0.8242, avg_rank = 16.2124, acc@5 =

Evaluation: mrr = 0.8184, avg_rank = 20.9484, acc@5 = 0.8769
/data/fan/UsersInOsakaProcessed/20130213_interp.pk
Evaluation: mrr = 0.8150, avg_rank = 20.8815, acc@5 = 0.8740
/data/fan/UsersInOsakaProcessed/20130214_interp.pk
Evaluation: mrr = 0.8147, avg_rank = 20.7119, acc@5 = 0.8751
/data/fan/UsersInOsakaProcessed/20130215_interp.pk
Evaluation: mrr = 0.8118, avg_rank = 21.0027, acc@5 = 0.8726
/data/fan/UsersInOsakaProcessed/20130216_interp.pk
Evaluation: mrr = 0.8102, avg_rank = 22.2445, acc@5 = 0.8682
/data/fan/UsersInOsakaProcessed/20130217_interp.pk
Evaluation: mrr = 0.8237, avg_rank = 19.6389, acc@5 = 0.8783
/data/fan/UsersInOsakaProcessed/20130218_interp.pk
Evaluation: mrr = 0.8221, avg_rank = 20.3917, acc@5 = 0.8793
/data/fan/UsersInOsakaProcessed/20130219_interp.pk
Evaluation: mrr = 0.8180, avg_rank = 20.6636, acc@5 = 0.8762
/data/fan/UsersInOsakaProcessed/20130220_interp.pk
Evaluation: mrr = 0.8121, avg_rank = 21.4605, acc@5 = 0.8724
/data/fan/UsersInOsakaProcessed/20130221_in

In [12]:
with open('./results_osaka/evaluation_local_rank_fl.pk', 'wb') as f:
    pk.dump(mrr_avg_rank_dict, f)